<a href="https://colab.research.google.com/github/seungjookang/aihub_news_labeling_project/blob/main/AIHub_News_Labeling_Preprocessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 0. import 및 코드 실행 전 필수 조건 설명

In [1]:
import os
import json
import zipfile
import requests
from tqdm import tqdm
from google.colab import drive
from google.colab import files
import shutil
import re

구글 코렙에 직접 다운로드 하려면 아래의 에러가 표시되기 때문에, WSL를 이용해 로컬 환경에 다운로드 후 구글 코렙에 업로드하는 방식을 선택했습니다.

```
Error msg:
AI 허브는 해외에서의 데이터 다운로드를 제한하고 있습니다.
```

때문에, WSL가 설치된 윈도우 환경이 본 ipynb 파일 실행을 위한 필수 조건이 되는 부분 양해 부탁드리겠습니다.



In [ ]:
# # 에러가 표시되는 코드입니다. 참고용으로 추가해 놓겠습니다.

# # aihubshell 다운로드
# !curl -o "aihubshell" https://api.aihub.or.kr/api/aihubshell.do

# # 실행 권한 부여
# !chmod +x aihubshell

# # 발급받은 API Key 입력
# api_key = '5B250C49-CCE8-4A0F-9C07-ED75211AAAAA' # 유효한 API Key로 변경하여 사용할 것.

# # 데이터셋 키와 라벨링 데이터의 파일 키 입력
# dataset_key = '71557'
# train_file_key = '470567'
# validation_file_key = '470569'

# # 라벨링 데이터 다운로드
# !./aihubshell -aihubapikey {api_key} -mode d -datasetkey {dataset_key} -filekey {train_file_key},{validation_file_key}

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  6336  100  6336    0     0  13666      0 --:--:-- --:--:-- --:--:-- 13655
aihubshell version 25.02.12 v0.5
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100    82  100    82    0     0    115      0 --:--:-- --:--:-- --:--:--   115
Download failed with HTTP status 502.
Error msg:
AI 허브는 해외에서의 데이터 다운로드를 제한하고 있습니다.


# 1. 로컬 환경에 라벨링 데이터 저장

In [2]:
# 아래 변수들은 각 환경별 적절한 변수들로 변경이 필요합니다.

# Windows 배경화면 경로 (WSL에서 Windows 경로로 접근)
WINDOWS_DESKTOP_PATH = "/mnt/c/Users/sammy/Desktop/AIHub_Data" # 유효한 로컬 디렉토리 주소로 변경하여 사용할 것.

# AI Hub API Key 및 데이터셋 정보
API_KEY = "5B250C49-CCE8-4A0F-9C07-ED75211D0DA3" # 유효한 API Key로 변경하여 사용할 것.

# 아래 변수들은 그대로 사용가능합니다.
DATASET_KEY = "71557"
TRAIN_FILE_KEY = "470567"
VALIDATION_FILE_KEY = "470569"

In [3]:
# WSL 실행용 Bash 스크립트 생성
bash_script = f"""#!/bin/bash

# Windows 배경화면 경로 (Colab에서 전달한 값 사용)
WINDOWS_DESKTOP_PATH="{WINDOWS_DESKTOP_PATH}"

# 배경화면 디렉토리 생성
mkdir -p "$WINDOWS_DESKTOP_PATH"
cd "$WINDOWS_DESKTOP_PATH"

# AIHUBSHELL 다운로드
curl -o "aihubshell" https://api.aihub.or.kr/api/aihubshell.do
chmod +x aihubshell

# AI Hub API Key 및 데이터셋 정보 (Colab에서 전달한 값 사용)
API_KEY="{API_KEY}"
DATASET_KEY="{DATASET_KEY}"
TRAIN_FILE_KEY="{TRAIN_FILE_KEY}"
VALIDATION_FILE_KEY="{VALIDATION_FILE_KEY}"

# AI Hub 데이터 다운로드
./aihubshell -mode d -datasetkey $DATASET_KEY -filekey $TRAIN_FILE_KEY,$VALIDATION_FILE_KEY -aihubapikey $API_KEY

echo "✅ AI Hub 데이터 다운로드 완료! Windows 배경화면에 저장되었습니다."
"""

# Colab에서 WSL 실행용 스크립트 저장
script_path = "/content/download_aihub.sh"
with open(script_path, "w") as f:
    f.write(bash_script)

print(f"✅ WSL 실행용 스크립트가 생성되었습니다! 파일 경로: {script_path}")

✅ WSL 실행용 스크립트가 생성되었습니다! 파일 경로: /content/download_aihub.sh


In [4]:
files.download("/content/download_aihub.sh")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# 2. 로컬에 저장된 스크립트를 이용해 AI Hub 데이터 다운로드

Windows에서 WSL을 실행하고, 다운로드된 스크립트를 실행합니다.

(1) Windows에서 WSL 실행

Windows 키 + R → wsl 입력 후 실행
아래 명령어를 입력하여 다운로드된 스크립트를 실행 가능하도록 이동합니다.
(디렉토리 주소를 각 환경에 유효한 디렉토리 주소로 변경해주세요.)

```
cd /mnt/c/Users/*****/Downloads
chmod +x download_aihub.sh
```

(2) WSL에서 AI Hub 데이터 다운로드 실행


```
bash download_aihub.sh
```

이제 AI Hub 데이터가 C:\Users\\*****\Desktop\AIHub_Data\(혹은 변경된 적절한 로컬 디렉토리 주소)에 다운로드됩니다.

(3) 다운로드된 데이터를 구글 드라이브에 업로드

구글 코렙에 가져와 데이터를 사용하기 위해 구글 드라이브에 데이터를 업로드 해줍니다.

# 3. 구글 드라이브에 업로드된 데이터를 코렙으로 가져와 압축 해제

In [5]:
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
# Google Drive 내 라벨링데이터 경로 (환경에 따라 적절한 디렉토리로 변경할 것.)
train_label_zip_path = "/content/drive/MyDrive/AIHub_Data/138.뉴스_대본_및_앵커_음성_데이터/01-1.정식개방데이터/Training/02.라벨링데이터/TL.zip"
val_label_zip_path = "/content/drive/MyDrive/AIHub_Data/138.뉴스_대본_및_앵커_음성_데이터/01-1.정식개방데이터/Validation/02.라벨링데이터/VL.zip"

# Colab 내 저장 경로
colab_train_label_zip_path = "/content/TL.zip"
colab_val_label_zip_path = "/content/VL.zip"

# Google Drive에서 Colab으로 복사
shutil.copy(train_label_zip_path, colab_train_label_zip_path)
shutil.copy(val_label_zip_path, colab_val_label_zip_path)

'/content/VL.zip'

In [7]:
# 압축 해제할 경로
TL_extract_path = "/content/TL"
VL_extract_path = "/content/VL"

# 압축 해제
with zipfile.ZipFile(colab_train_label_zip_path, 'r') as zip_ref:
    zip_ref.extractall(TL_extract_path)
with zipfile.ZipFile(colab_val_label_zip_path, 'r') as zip_ref:
    zip_ref.extractall(VL_extract_path)

# 4. 압축 해제한 json 파일들을 txt 파일로 변환

In [8]:
# TL 및 VL 디렉토리 설정
label_dirs = ["/content/TL", "/content/VL"]

# 저장할 파일 경로
text_only_path = "/content/text_only.txt"
text_preprocessed_path = "/content/text_preprocessed.txt"

# 전처리 함수: 괄호 안의 왼쪽/오른쪽 내용을 분리하여 출력
def preprocess_text(text):
    # 정규식 패턴: "(왼쪽)/(오른쪽)" 형태 탐색
    matches = re.findall(r"\(([^)]+)\)/\(([^)]+)\)", text)

    # 왼쪽 괄호 내용과 오른쪽 괄호 내용으로 분리
    text_left = text
    text_right = text

    for left, right in matches:
        text_left = text_left.replace(f"({left})/({right})", left)  # 왼쪽 괄호 내용 유지
        text_right = text_right.replace(f"({left})/({right})", right)  # 오른쪽 괄호 내용 유지

    return f"{text_left} ${text_right}"


# 파일 쓰기 준비
with open(text_only_path, "w", encoding="utf-8") as text_only_file, \
     open(text_preprocessed_path, "w", encoding="utf-8") as text_preprocessed_file:

    json_count = 0  # JSON 파일 개수 카운트

    # TL & VL 디렉토리 모두 탐색 (디렉토리 및 파일명을 정렬하여 순서 유지)
    for label_root in sorted(label_dirs):  # TL → VL 순서 유지
        for spk_folder in sorted(os.listdir(label_root)):  # SPK003 ~ SPK089 정렬
            spk_path = os.path.join(label_root, spk_folder)
            if not os.path.isdir(spk_path):
                continue  # 폴더가 아니면 무시

            for sub_folder in sorted(os.listdir(spk_path)):  # 하위 폴더 정렬
                sub_path = os.path.join(spk_path, sub_folder)
                if not os.path.isdir(sub_path):
                    continue  # 폴더가 아니면 무시

                for file in sorted(os.listdir(sub_path)):  # 파일명 순서대로 정렬
                    if file.endswith(".json"):
                        json_path = os.path.join(sub_path, file)

                        # JSON 파일 읽기
                        try:
                            with open(json_path, "r", encoding="utf-8") as f:
                                data = json.load(f)

                            # script.text 키가 존재하는지 확인
                            if "script" in data and "text" in data["script"]:
                                transcript = data["script"]["text"]
                            else:
                                print(f"⚠️ script.text 키 없음! 파일: {json_path}")
                                continue

                            # JSON 파일 이름에서 확장자 제거하여 audio_name 생성
                            audio_name = os.path.splitext(file)[0]

                            # 원본 텍스트 저장
                            text_only_file.write(f"{audio_name}:{transcript}\n")

                            # 전처리 후 텍스트 저장
                            processed_text = preprocess_text(transcript)
                            text_preprocessed_file.write(f"{audio_name}:{processed_text}\n")

                            json_count += 1  # 정상 처리된 JSON 파일 개수 증가
                        except Exception as e:
                            print(f"❌ 오류 발생: {json_path} | {e}")

print(f"✅ TL & VL 전사 텍스트 및 전처리 데이터 저장 완료! (총 {json_count}개 파일 처리됨)")


✅ TL & VL 전사 텍스트 및 전처리 데이터 저장 완료! (총 341860개 파일 처리됨)


In [9]:
# txt파일 내용 확인용 코드
!head /content/text_only.txt
print()
!head /content/text_preprocessed.txt

SPK003KBSEC032F001:한 회사에서 무려 (80)/(팔십) 년 넘게 근무한 브라질 남성이 세계에서 직장생활을 가장 오래 한 사람으로 기네스북에 이름을 올렸습니다.
SPK003KBSEC032F002:이 신기록의 주인공은 브라질의 한 의류 원단 회사에 다니는 바우테르 오르트만 씨입니다.
SPK003KBSEC032F003:(12)/(열 두) 살 때부터 돈을 벌기 위해 생활 전선에 뛰어든 그는 열다섯 살 때인 (1938)/(천 구백 삼십 팔) 년, 이 회사에 정식으로 취직한 뒤 (100)/(백) 세가 된 지금까지 이곳에서만 (84)/(팔십 사) 년째 현역으로 근무 중입니다.
SPK003KBSEC032F004:여전히 주 (5)/(오) 일 출근을 유지하는 오르트만 씨는 '처음 일을 시작할 때와 비교하면 세상이 엄청나게 달라졌다' 고 말하며 요즘은 스마트폰과 태블릿으로 거의 모든 업무를 처리한다고 밝혔는데요.
SPK003KBSEC032F005:이어, 건강을 유지하면서 직장생활을 오래 할 수 있는 비결을 묻는 질문에 '내일을 걱정하지 말고 오늘을 열심히 살아야 한다' 면서 '조바심 내지 말고 느긋하게 웃으며 사는 것이 비결' 이라고 강조했습니다.
SPK003KBSIN028F001:한 해 숨지는 교통사고 사망자 수가 계속 감소해, (3)/(삼) 천 명대로 떨어졌습니다.
SPK003KBSIN028F002:사망자가 줄어든 건 반가운 일이지만, 보행자 안전은 여전히 (OECD)/(오 이 씨 디) 꼴찌 수준이어서 대책 마련이 시급한 것으로 나타났습니다.
SPK003KBSIN028F003:지난해 교통사고로 숨진 사람이 (3,700)/(삼천 칠백) 여 명으로 집계됐습니다.
SPK003KBSIN028F004:사망자가 (3)/(삼) 천명 대로 내려온 것은 (42)/(사십 이) 년 만입니다.
SPK003KBSIN028F005:음주운전 처벌이 강화되면서 음주 교통사고 사망자가 (20%)/(이십 퍼센트) 넘게 감소한 것이 주요 원인으로 꼽힙니다.

SPK003KBSEC032F001:한

In [10]:
# txt파일 내용 확인용 코드
!tail /content/text_only.txt
print()
!tail /content/text_preprocessed.txt

SPK089YTNSO933M001:이용수 할머니가 다음 달 정의기억연대에서 주최하는 수요집회에 참석할 계획입니다.
SPK089YTNSO933M002:정의연 관계자는 이용수 할머니가 다음 달 (12)/(십 이) 일에 열리는 `제 (8)/(팔) 차 일본군위안부 기림일 세계연대집회 기자회견`에 참석할 예정이라고 밝혔습니다.
SPK089YTNSO933M003:정의연 측은 이용수 할머니와 여러 차례 만나 그동안 있었던 오해를 풀고 난 뒤 할머니가 참석한다는 결정을 내렸다고 전했습니다.
SPK089YTNSO933M004:일본군 위안부 피해자인 이 할머니는 지난 (1)/(일) 일과 (3)/(삼) 일 이나영 정의기억연대 이사장과의 자리에서 수요시위에 참여하겠다고 밝힌 것으로 전해졌습니다.
SPK089YTNSO933M005:종로구가 옛 일본대사관 앞에 있는 평화의 소녀상 부근에서 집회와 시위를 금지하면서, 다음 달 (12)/(십 이) 일 수요집회는 기자회견 방식으로 진행될 계획입니다.
SPK089YTNSO934M001:정부가 재입국하는 외국인을 입국 심사할 때, 거주지가 실제 거주하는 곳이 맞는지와 자가격리에 적합한 장소인지 등을 확인하기로 했습니다.
SPK089YTNSO934M002:이는 최근 장기체류 외국인이 출국 후 재입국할 때 체류 예정지라고 신고한 곳이 자가격리에 부적합한 경우가 발생함에 따라 자가격리 관리를 보다 강화하기 위한 것입니다.
SPK089YTNSO934M003:중앙재난안전대책본부는 코로나 (19)/(일 구) 확진자가 급증하는 등 검역 관리가 필요한 국가에서 방역 관리상 취약한 유형의 체류자격으로 입국하는 외국인을 대상으로 자가격리 장소 확인 절차를 보다 엄격하게 운영하기로 했습니다.
SPK089YTNSO934M004:여러 사람이 공동으로 숙식하거나 화장실을 공유하는 등 주거의 독립성이 없는 쪽방촌 등에 체류하는 경우에는 입국자를 자가격리에서 시설 격리로 전환할 방침입니다.
SPK089YTNSO934M005:국내 체류 중인 등록외국인의 체류지 관리를 강화하

In [11]:
# JSON 파일 개수 확인용 코드입니다.

# JSON 파일 개수 저장용 변수
total_count = 0

# TL & VL 디렉토리 모두 탐색
for label_root in label_dirs:
    json_count = sum(len(files) for _, _, files in os.walk(label_root) if any(f.endswith(".json") for f in files))

    print(f"📂 {label_root} 폴더 내 JSON 파일 개수: {json_count}")
    total_count += json_count

# 전체 파일 개수 출력
print(f"✅ TL + VL 총 JSON 파일 개수: {total_count}")

📂 /content/TL 폴더 내 JSON 파일 개수: 306734
📂 /content/VL 폴더 내 JSON 파일 개수: 35126
✅ TL + VL 총 JSON 파일 개수: 341860
